In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
root_path = '/content/drive/MyDrive/Database/5153/'
import os
os.chdir(root_path)

## Results combined and Emotion vectorization
Weighted average by probability

In [22]:
map = pd.read_csv('map.csv')
map

,Mood,Valence,Arousal
0,ANGER,2.81,5.03
1,FEAR,2.59,4.33
2,HAPPY,5.37,4.99
3,SAD,5.05,2.55
4,SURPRISE,3.10,4.46
5,TENDER(Neutral）,5.93,2.89
6,HIGH ENERGY,4.14,5.67
7,HIGH VALENCE,5.68,4.19
8,LOW ENERGY,4.79,2.31
9,LOW VALENCE,1.81,4.62


In [23]:
## Rename
map.loc[5,'Mood'] = 'neutral'
map.loc[0,'Mood']=  'angry'
map.Mood = map.Mood.map(lambda x: x.lower())
map = map.iloc[:6,:]

In [3]:
# Input audio model results
audio_data = pd.read_csv('audio-recommendation.csv')
audio_data.head()

,angry,fear,happy,neutral,sad,surprise,pred_label,label
0,0.000466,0.028949,0.007229,0.386752,0.554057,0.022548,sad,surprise
1,0.569293,0.000263,0.371317,0.056847,0.000176,0.002104,angry,angry
2,0.998569,0.001125,0.000134,0.000082,0.000009,0.000081,angry,angry
3,0.013970,0.000781,0.008347,0.000468,0.000170,0.976264,surprise,angry
4,0.002399,0.001072,0.004207,0.057186,0.934446,0.000690,sad,sad


In [87]:
# Input facial model results
# Unify column names
facial_data = pd.read_csv('facial-recommendation.csv')
facial_data.rename(columns = {'emotion': 'label', 'happiness':'happy','anger':'angry','sadness':'sad'}, inplace = True)

In [90]:
facial_data.head()

,label,angry,fear,happy,sad,surprise,neutral,pred_label
0,anger,0.825429,0.023492,0.037990,0.054240,0.000671,0.058178,anger
1,sadness,0.764434,0.025851,0.001584,0.174339,0.001903,0.031888,anger
2,neutral,0.020967,0.086415,0.054299,0.077410,0.001491,0.759418,neutral
3,happiness,0.000063,0.000337,0.997493,0.000037,0.002006,0.000064,happiness
4,happiness,0.001850,0.000423,0.890872,0.001774,0.000918,0.104163,happiness


In [31]:
def get_valence_arousal(data, map):
  '''
  Input:
    data: User emotion prediction output probability vector including 6 dimensions (angry	fear	happy	neutral	sad	surprise)
          Togther with two additional features: pred_label, label.
    map: Valence-arousal score for six emotions
  Output:
    Dataframe contains valence-arousal scores.
    Features: Valence, Arousal,	pred_label, label
  '''
  map_dict = dict.fromkeys(map.Mood)
  for key in map_dict.keys():
    map_dict[key] = np.array(map.loc[map['Mood'] == key, ['Valence','Arousal']])
  ## Prepare emotion Valence-arousal vector and probability matrix
  emotion = map_dict.keys()
  data_m = data.loc[:,emotion]
  emotion_v = np.array([i[0] for i in map_dict.values()])
  data_score = data_m @ emotion_v
  data_score_df = pd.DataFrame(data_score.values, columns = ['Valence','Arousal'])
  data_score_df['pred_label'] = data['pred_label']
  data_score_df['label'] = data['label']
  return data_score_df

In [92]:
user_emotion = get_valence_arousal(data=facial_data, map=map)
user_emotion

,Valence,Arousal,pred_label,label
0,3.205295,4.752637,anger,anger
1,3.298931,4.510154,anger,sadness
2,5.473210,3.149356,neutral,neutral
3,5.364372,4.988493,happiness,happiness
4,5.419768,4.766237,happiness,happiness
...,...,...,...,...
3528,4.997348,2.631197,sadness,sadness
3529,5.369557,4.989769,happiness,happiness
3530,5.283287,2.850115,sadness,sadness
3531,4.398985,3.415920,sadness,sadness


## Music recommend based on Valence-arousal score
Calculate similarty betweem user score and all music valence-arousal score 

In [154]:
music_df_1 = pd.read_csv('mean_ratings_set1.csv')
music_tracklist = pd.read_csv('set1_tracklist.csv')
music_tracklist.rename(columns = {'Nro':'Number'},inplace = True)
music_df_1.head()

,number,valence,energy,tension,anger,fear,happy,sad,tender,TARGET
0,1,4.83,6.83,3.17,1.00,1.0,7.33,1.00,1.00,HAPPY
1,2,4.83,6.17,3.83,1.00,1.0,7.17,1.17,1.83,HAPPY
2,3,5.60,6.20,3.00,1.00,1.0,7.17,1.00,1.00,HAPPY
3,4,4.67,6.33,3.83,1.00,1.0,7.17,1.00,1.00,HAPPY
4,5,6.50,5.00,3.17,1.17,1.0,7.17,1.00,2.83,HAPPY


In [155]:
# Filter music with target emotions we want
music_df_1 = music_df_1[music_df_1['TARGET'].isin(['HAPPY', 'SAD', 'TENDER', 'FEAR', 'ANGER', 'SURPRISE'])]

Energy equals to arousal in our case.

In [13]:
from numpy import dot
from numpy.linalg import norm
def cos_similarity(a,b): 
  return dot(a, b)/(norm(a)*norm(b))

In [156]:
def sample_recommend(user_emotion, music_dataset, music_tracklist, top = 3):
  '''
  Input:
    user_emotion: valence_arousal score for one record, 2D list-like
    music_dataset: music dataset with columns, Number, Valence, Energy at least
    music_tracklist: music list with number as index, is used for sourcing details of mucic recommended
    top: number of top similar music recommended
  Output:
    Dataframe contains topN music detailed information
  '''
  music_dataset = music_dataset.copy()
  music_dataset['similarity'] = music_dataset.apply(lambda x: cos_similarity(user_emotion, x[['valence','energy']]), axis = 1)
  top_music = music_dataset.sort_values('similarity', ascending = False).iloc[:top,:]
  top_num = top_music['number']
  results = music_tracklist.loc[music_tracklist['Number'].isin(top_num), :]
  return top_music, results

### Testing our recommender

In [150]:
def test(user_num, top=3):
  '''
  Input:
    user_num: user number, starts from 0
    top
  Output: testing results
  '''
  user_emotion_score = user_emotion.iloc[user_num,:2]
  music, results = sample_recommend(user_emotion_score, music_df_1, music_tracklist, top = top)
  combined_results = pd.merge(music[['number','valence','energy','similarity']], results,  left_on = 'number', right_on = 'Number')
  combined_results.drop('Number', axis = 1, inplace = True)
  print('User\'s valence-rousal score:\n{}\n\nPredictied emotion: {}'.format(user_emotion.loc[user_num,['Valence','Arousal']], 
                                                                         user_emotion.loc[user_num,'pred_label']))
  print('\nTop{} Recommend music:'.format(top))
  print('#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-')
  print(combined_results)
  print('#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-')

In [157]:
test(3)

User's valence-rousal score:
Valence    5.364372
Arousal    4.988493
Name: 3, dtype: object

Predictied emotion: happiness

Top3 Recommend music:
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
   number  valence  energy  similarity   Emotion           Album name  Track  \
0      27     4.60    4.20    0.999958     Happy        Shallow Grave      8   
1     153     3.83    3.67    0.999888  Surprise          Naked Lunch     14   
2      22     5.60    5.00    0.999795     Happy  Shakespeare in Love     21   

       Min:Sec  
0  00:00-00:15  
1  01:02-01:17  
2  00:03-00:21  
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-


In [ ]:
import matplotlib.pyplot as plt
